# Maximizing Parameters

This network provides some various hyperparameters that affect the optimization. This code finds these optimal parameters by assessing the quality and speed of optimization with these parameters

In [0]:
import tensorflow as tf
import math
import itertools
import numpy as np
import random
import matplotlib.pyplot as plt

# tf.enable_eager_execution()


data_size = 150


USING_MNIST = False
x_train = []

for i in range(data_size):
  x = round(math.sqrt(random.randint(0,4095)))
  x_train.append([(x//32)%2,(x//16)%2,(x//8)%2,(x//4)%2,(x//2)%2,x%2])
#     x_train.append([0 if .5<random.random() else 1 for k in range(4)])
# goodlist = list(map(lambda x: list(x), itertools.product([0,1], repeat=N+1)))
# x_train = np.array(goodlist)
s = tf.dtypes.cast(tf.constant(x_train),dtype=tf.float32)
s = tf.reshape(s,[data_size,-1])


In [2]:
N = 3
# K = .3
# L = .8
# MU = 2

s_size = s.shape[1]

#creating an array of the permutations to aid in later computation of H(Y^0...Y^N)
def get_options():
  bin_options = list(itertools.product([0,1], repeat=N+1))
  options = []
  for row_index, row in enumerate(bin_options):
    row_building = []
    for index,val in enumerate(row):
      row_building.append(index*2+val)
    options.append(row_building)
  return options
options = get_options()



#weights
x_individual = [tf.Variable(tf.random_normal(shape=[int(s.shape[1]),1], stddev=0.1),constraint=lambda t: tf.clip_by_value(t, -4, 4)) for i in range(N+1)]
x = tf.concat(x_individual,axis=1)

# x = tf.Variable(tf.random_normal(shape=[int(s.shape[1]),N+1], stddev=0.1))
# B = tf.Variable(tf.random_normal(shape=[N+1], stddev=0.1))

B_individual = [tf.Variable(tf.random_normal(shape = [1], stddev=0.1),constraint=lambda t: tf.clip_by_value(t, -4, 4)) for i in range(N+1)]
B = tf.concat(B_individual,axis=0)
print(B)

Tensor("concat_1:0", shape=(4,), dtype=float32)


In [0]:
#@title
def compute_nodes(s):
#   output = conv_net(s,N+1,.25,False,True)
  layer = tf.matmul(s,x)+B
  return tf.nn.sigmoid(layer)
#   layer_clip = tf.clip_by_value(layer,-4,4)
#   return tf.clip_by_value(tf.nn.sigmoid(layer_clip),.01,.99)



def compute_Hs(computed_nodes):
  one_over_num_samples = 1/tf.dtypes.cast(computed_nodes.shape[0],dtype=tf.float32)
  return -tf.math.log(one_over_num_samples)/math.log(2)
  

#p(y^k)
def compute_pyk(computed_nodes):
   return tf.math.reduce_mean(computed_nodes,axis=0)#CHECKED
  
def compute_Hs_y(computed_nodes):
  num_samples = tf.dtypes.cast(computed_nodes.shape[0],dtype=tf.float32)
  prob_y_and_s = (1./num_samples)*computed_nodes
  prob_y_and_not_s = (1./num_samples)*(1-computed_nodes)
  logedvals = prob_y_and_s*tf.math.log(prob_y_and_s)/math.log(2) + prob_y_and_not_s*tf.math.log(prob_y_and_not_s)/math.log(2)
  return -tf.math.reduce_sum(logedvals,axis=0)

#H(Y^k)
#kth elemenent of returned value = H(Y^k)
def compute_Hy_k(computed_nodes):
  pyk = compute_pyk(computed_nodes)
  logednodes_1 = tf.math.log(pyk)/math.log(2)
  logednodes_2 = tf.math.log(1-pyk)/math.log(2)
  
  return -logednodes_1*pyk - logednodes_2*(1-pyk) #CHECKED


#H(Y^0,...,Y^N)
def compute_Hy(computed_nodes):
  tf_options = tf.constant(options, dtype=tf.float32)
  computed_nodes_both = [computed_nodes,1-computed_nodes]
  
  def process_image(row):
    odds_both = [row,1-row]
    def fnc(y):
      def inner_func(x):
        x = tf.dtypes.cast(x,dtype=tf.int32)
        return tf.gather(tf.gather(odds_both,x%2),x//2)
      return tf.map_fn(inner_func, y)
    '''
      vals is 2^(N+1) by N, where the [i][j] entry is probability that
      in the ith permutation, say 0101,
      (y^0=0, y^1=1, y^2=0, y^3=1)
    '''
    vals = tf.map_fn(fnc,tf_options)

    reduced = tf.math.reduce_prod(vals,axis=1)

    return reduced
  
  #p(y^0,y^1,...) length 2^n+1
  pys = tf.math.reduce_mean(tf.map_fn(process_image,computed_nodes),axis=0)
  return - tf.math.reduce_sum(pys*tf.math.log(pys)/math.log(2)) #Checked


#computer H(Y^i,Y^j)
def compute_H_2(computed_nodes):
  def sub_compute(computed_nodes_1, computed_nodes_2):
    def top(x):
      return tf.reshape(tf.tile(x,[x.shape[0]]),[x.shape[0],x.shape[0]])
    def bot(x):
      return tf.transpose(tf.reshape(tf.tile(x,[x.shape[0]]),[x.shape[0],x.shape[0]]))
    pairwise1 = tf.map_fn(bot,computed_nodes_1)
    pairwise2 = tf.map_fn(top,computed_nodes_2)
    prob_a_b = tf.math.reduce_mean(pairwise1*pairwise2,axis=0) #reduce mean across sample size
#     print(prob_a_b[0][1]*math.log(prob_a_b[0][1])/math.log(2))
#     print("\nprob a b:\n{}\n".format(prob_a_b))
    return prob_a_b*tf.math.log(prob_a_b)/math.log(2)


  info = sub_compute(computed_nodes, computed_nodes)+\
          sub_compute(1-computed_nodes, computed_nodes)+\
          sub_compute(computed_nodes, 1-computed_nodes)+\
          sub_compute(1-computed_nodes, 1-computed_nodes)
  return -info*(1-tf.eye(N+1,N+1))





def compute_utility_2(computed_nodes, mu, lamb, kappa):
#   print("Hs:\n{}\n".format(compute_Hs(computed_nodes)))
#   print("Hs_y:\n{}\n".format(compute_Hs_y(computed_nodes)))
#   print("Hy_k:\n{}\n".format(compute_Hy_k(computed_nodes)))
#   print("Hy2:\n{}\n".format(compute_H_2(computed_nodes)))
  term1 = mu*(compute_Hs(computed_nodes)-compute_Hs_y(computed_nodes))
  hyk = compute_Hy_k(computed_nodes)
  term2 = lamb*N*hyk
  term3 = -kappa*(tf.reduce_sum(hyk)-hyk)
  term4 = (lamb-kappa)*tf.reduce_sum(compute_H_2(computed_nodes),axis=0)
  return term1+term2+term3+term4
  
 

In [4]:
#@title Default title text { run: "auto", vertical-output: true, display-mode: "form" }
def get_cn2():
  return tf.constant([[.01 if random.random()<.5 else .99 for i in range(N+1)] for k in range(len(options))])
cn3 = tf.constant([[.5 for i in range(N+1)] for k in range(len(options))])
goodlist = list(map(lambda x: list(x), itertools.product([.01,.99], repeat=N+1)))
cn1 = tf.constant(goodlist)
cn4 = tf.constant([[.01,.01,.01,.01] for k in range(len(options))])



u1 = compute_utility_2(cn1,1,.10+.1,.1)
print("U1:{}".format(u1))
u2 = compute_utility_2(get_cn2(),1,.10+.1,.1)
print("U2:{}".format(u2))
u3 = compute_utility_2(cn3,1,.10+.1,.1)
print("U3:{}".format(u3))
u4 = compute_utility_2(cn4,1,.10+.1,.1)
print("U4:{}".format(u4))
print("\n")
print("U1-U2: {}".format(sum(u1)-sum(u2)))
print("U1-U3: {}".format(sum(u1)-sum(u3)))
print("U1-U4: {}".format(sum(u1)-sum(u4)))

# even_replica_list = []
# uneven_replica_list = []
# for i in range(len(goodlist)):
#   for k in range(5):
#   even_replica_list.append(goodlist[i])
#   uneven_replica_list.append(goodlist[i])
# for reps in range(len(goodlist)*4):
#   index = random.randint(0, len(goodlist)-1)
#   uneven_replica_list.append(goodlist[index])

# print(uneven_replica_list)
# print(len(even_replica_list))
# cn1 = tf.constant(even_replica_list)
# cn2 = tf.constant(uneven_replica_list)

# # # print(c)
if False:
  import pylab as plt
  import numpy as np

  # Sample data
  side = np.linspace(-1,4,10,10)
  lambda_plot,kappa_plot = np.meshgrid(side,side)
  Z = np.zeros((len(lambda_plot),len(lambda_plot[0])))
  Z_all = [np.zeros((len(lambda_plot),len(lambda_plot[0]))) for sfdjk in range(5)]
  for x_p in range(len(lambda_plot)):
    print(x_p)
    for y_p in range(len(lambda_plot[0])):
        l = lambda_plot[x_p][y_p]
        k = kappa_plot[x_p][y_p]
        
        u1 = compute_utility_2(cn1,1,l,k)
        Z_all[1][x_p][y_p] = sum(u1)
        
        u2 = compute_utility_2(get_cn2(),1,l,k)
        Z_all[2][x_p][y_p] = sum(u2)
        
        u3 = compute_utility_2(cn3,1,l,k)
        Z_all[3][x_p][y_p] = sum(u3)
        
        u4 = compute_utility_2(cn4,1,l,k)
        Z_all[4][x_p][y_p] = sum(u4)
        
        Z[x_p][y_p] = 3*abs(sum(u2))-(abs(sum(u1))+abs(sum(u3))+abs(sum(u4)))

  # Plot the density map using nearest-neighbor interpolation
  plt.pcolormesh(lambda_plot,kappa_plot,Z)
  plt.show()

U1:Tensor("add_6:0", shape=(4,), dtype=float32)
U2:Tensor("add_13:0", shape=(4,), dtype=float32)
U3:Tensor("add_20:0", shape=(4,), dtype=float32)
U4:Tensor("add_27:0", shape=(4,), dtype=float32)




TypeError: ignored

In [0]:
#@title
kappa_lin = np.linspace(-20,40,100)
lambda_lin = list(map(lambda x:x+.108, kappa_lin))
print(lambda_lin)
loger = []
for i in range(len(kappa_lin)):
  l = lambda_lin[i]
  k = kappa_lin[i]
  
  u1 = compute_utility_2(cn1,1,l,k)
  u2 = sum([sum(compute_utility_2(get_cn2(),1,l,k)) for dfs in range(8)])/8.0
  loger.append(float(sum(u1)-u2))
plt.plot(loger)
print()

In [0]:
#@title
kappa_lin = np.linspace(-5,2,20)
u1u2 = []
u1u3 = []
u1u4 = []
for i in range(len(kappa_lin)):
  l = 0
  k = kappa_lin[i]
  u1 = compute_utility_2(cn1,1,l,k)
  u2 = sum([sum(compute_utility_2(get_cn2(),1,l,k)) for sfd in range(7)])/7.0
  u3 = compute_utility_2(cn3,1,l,k)
  u4 = compute_utility_2(cn4,1,l,k)
#   print("U4:{}".format(u4))
#   print("\n")
  u1u2.append(sum(u1)-u2)
  u1u3.append(sum(u1)-sum(u3))
  u1u4.append(sum(u1)-sum(u4))

plt.plot(kappa_lin,u1u2,label="u1-u2")
plt.plot(kappa_lin,u1u3,label="u1-u3")
plt.plot(kappa_lin,u1u4,label="u1-u4")
plt.plot(kappa_lin,np.zeros(20),label="0")
plt.plot(kappa_lin,u1u3,label="u1-u3")
plt.legend()
plt.show()

In [0]:
#@title
plt.plot(kappa_lin,np.log(u1u2),label="u1-u2")
plt.plot(kappa_lin,np.log(u1u3),label="u1-u3")
plt.plot(kappa_lin,np.log(u1u4),label="u1-u4")
# plt.plot(kappa_lin,np.log(np.zeros(100)),label="0")
plt.legend()
plt.show()

In [0]:
#@title
plt.plot(kappa_lin,loger)
plt.show()

In [0]:
#@title
# for index in range(1,5):
#   fig, (ax0) = plt.subplots(ncols = 1)
#   im = ax0.pcolormesh(lambda_plot,kappa_plot,Z_all[index])
#   fig.colorbar(im, ax=ax0)
#   ax0.set_title("Utility {}".format(index))

Z_good = np.zeros((len(lambda_plot),len(lambda_plot[0])))
for x_p in range(len(lambda_plot)):
  for y_p in range(len(lambda_plot[0])):
      l = lambda_plot[x_p][y_p]
      k = kappa_plot[x_p][y_p]
      Z_good[x_p][y_p] = Z_all[1][x_p][y_p] - Z_all[4][x_p][y_p]
#       print(Z_good[x_p][y_p])
fig, (ax0) = plt.subplots(ncols = 1)
im = ax0.pcolormesh(lambda_plot,kappa_plot,Z_good)
fig.colorbar(im, ax=ax0)
ax0.set_title("Utility 1-Utility 4")

In [0]:
#@title
maxx = 0
maxy = 0
maxval = 0
for x_p in range(len(lambda_plot)):
  for y_p in range(len(lambda_plot[0])):
#       print("{}, {}, {}, {}".format(1,lambda_plot[x_p][y_p],kappa_plot[x_p][y_p],Z[x_p][y_p]))
      val = Z[x_p][y_p]
#       if val>6:
#         print("good {} {}".format(lambda_plot[x_p][y_p],kappa_plot[x_p][y_p]))
      if lambda_plot[x_p][y_p]>0 and val>maxval:
        maxx, maxy = x_p,y_p
        maxval = val
print(maxx)
print(maxy)
print(lambda_plot[0][maxy])
print(kappa_plot[maxx][maxy])
print(maxval)


In [0]:
#@title
def get_x_numb(x):
  return int(8*float(x[0])+4*float(x[1])+2*float(x[2])+float(x[3]))
def get_output(x):
#   print(type(x))
  output2 = map(lambda y:str(int(round(float(y)))),x)
  return str("".join(output2))
def get_counts(key_list):
    countdict = {}
    for outcome in key_list:
      if outcome not in countdict:
        countdict[outcome] = 1
      else:
        countdict[outcome] += 1
    return countdict

def get_info(output):

  x_train_reshaped = x_train
  if USING_MNIST:
    x_train_reshaped = x_train.reshape(data_size,784)

  outcomes_as_keys = list(map( get_output,output))
  inputs_as_keys = list(map(get_output,x_train_reshaped))
  
  count_dict_outcome = get_counts(outcomes_as_keys)
  count_dict_input = get_counts(inputs_as_keys)
  
  input_output = {}
  for i in range(len(inputs_as_keys)):
    input_output[inputs_as_keys[i]] = outcomes_as_keys[i]
  
#   print(odds_image)


  
  mutual_info = 0
  number_samples = float(len(output))
  #a
  for input_str, input_odds in count_dict_input.items():
    output_str = input_output[input_str]
    output_odds= count_dict_outcome[output_str]
    mutual_info += input_odds/number_samples * math.log(1/(output_odds/number_samples))/math.log(2)
    
  
  return mutual_info

In [0]:
#@title

def get_information_no_tf(computed_nodes, mu, lamb, kappa):
  computed_nodes_tf = computed_nodes
  
  computed_nodes = list(computed_nodes)
  computed_nodes_np = np.array(computed_nodes)
  

  num_images = float(len(computed_nodes))
  
  answer = [0]*len(computed_nodes[0])

  for row in computed_nodes:
    for node_index, val in enumerate(row):

      answer[node_index] += float(mu*(1./num_images)*val*math.log(val)/math.log(2))
      val2 = 1-val
      answer[node_index] += float(mu*(1./num_images)*val2*math.log(val2)/math.log(2))


 
  def get_conditionals(computed_nodes1,computed_nodes2):
    pyk = compute_pyk(computed_nodes2)
    conditional_probabilities = [[0 for i in range(N+1)] for k in range(N+1)]
    for i in range(N+1):
        for k in range(N+1):
          if i == k:
            continue
          sumer = 0.
          count = 0.
          for im in range(len(computed_nodes)):
            sumer += computed_nodes1[im][i]*computed_nodes2[im][k]
            count += 1.
          conditional_probabilities[i][k] = float((sumer/count)/pyk[k])
    return np.array(conditional_probabilities)
  
  

  conditional_probabilities = [get_conditionals(computed_nodes_np,computed_nodes_np), get_conditionals(1-computed_nodes_np,computed_nodes_np), 
                               get_conditionals(computed_nodes_np,1-computed_nodes_np), get_conditionals(1-computed_nodes_np,1-computed_nodes_np)]

  
  
  print_dictionairy = {0:"p(s{},y^{}=1,y^{}=1)",1:"p(s{},y^{}=2,y^{}=1)",2:"p(s{},y^{}=1,y^{}=2)",3:"p(s{},y^{}=2,y^{}=2)"}
  print_dictionairy_conditional = {0:"p(y^{}=1 | y^{}=1)",1:"p(y^{}=2 | y^{}=1)",2:"p(y^{}=1 | y^{}=2)",3:"p(y^{}=2 | y^{}=2)"}

  for i_index, image in enumerate(computed_nodes):
    for j, val_j_original in enumerate(image):
      for k, val_k_original in enumerate(image):
        if j == k:
          continue
        for value_index in range(4):
          val_j = val_j_original
          if value_index == 1 or value_index == 3:
            val_j = 1-val_j_original
          val_k = val_k_original
          if value_index == 2 or value_index == 3:
            val_k = 1-val_k_original

          
          addme =  float(lamb*(1./num_images)*val_j*val_k)*math.log(conditional_probabilities[value_index][j][k])/math.log(2)
#           if k == 0:
#             print(print_dictionairy[value_index].format(i_index,j,k) + "* log(" + print_dictionairy_conditional[value_index].format(j,k) + \
#                   ") = " + str(float((1./num_images)*val_j*val_k)) + " * log(" + str(conditional_probabilities[value_index][j][k]) + ") = " + str(addme))
          
          answer[k] += addme
 

  for i_index, image in enumerate(computed_nodes):
    for j, val_j_original in enumerate(image):
      for k, val_k_original in enumerate(image):
        if j == k:
          continue
        for value_index in range(4):
          val_j = val_j_original
          if value_index == 1 or value_index == 3:
            val_j = 1-val_j_original
          val_k = val_k_original
          if value_index == 2 or value_index == 3:
            val_k = 1-val_k_original
          
          answer[k] -= float(kappa*(1./num_images)*val_j*val_k)*math.log(conditional_probabilities[value_index][k][j])/math.log(2)
        
  return answer


# print(compute_Hs(computed_nodes))
# print(compute_Hs_y(computed_nodes))
# print(compute_Hy_k(computed_nodes))
# print(compute_H_2(computed_nodes))
# print(compute_Hy(computed_nodes))

# print("\nH(s):\n{}".format(compute_Hs(computed_nodes)))
# print("H(s,y^k):\n{}\n".format(compute_Hs_y(computed_nodes)))
# print("Real")
# print(get_information_no_tf(computed_nodes, 1, 2, 1))
# print("tf")
# computed_nodes = tf.constant([[0.75, 0.25, 0.5],[0.1, 0.9, 0.5]])
# computed_nodes2 = tf.constant([[0.99, 0.01, 0.5],[0.01, 0.99, 0.5]])
# computed_nodes3 = tf.constant([[1, 0, 0.01],[0.01, 0.99, 0.99]])

# print(compute_utility_2(computed_nodes,3,1,2))
# print(compute_utility_2(computed_nodes2,3,1,2))
# print(compute_utility_2(computed_nodes3,3,1,2))
# print(N*compute_Hy_k(computed_nodes))


In [0]:
def get_set_string(numset):
  summation = 0
  for value in numset:
    summation += c2[value]
  return summation
def dotheprint(output):
  
  

  numbers_to_output = {}
  for i in range(len(x_train)):
    if USING_MNIST:
      num = y_train[i]
    else:
      num = x_train[i][0]*32+x_train[i][1]*16+x_train[i][2]*8+x_train[i][3]*4+x_train[i][4]*2+x_train[i][4]
  #     print(num)

    o = list(map(lambda x: int(x),list(get_output(output[i]))))
    
    o = 8*o[0]+4*o[1]+2*o[2]+o[3]
#     print(o)
#     print(o)
    num,o=o,num
  #   print(o)
  #   print("{}->{}".format(num,output))
    tmp0 = numbers_to_output.get(num,[])
    tmp0.append(o)
    numbers_to_output[num] = s
  for i in range(16):
    if i in numbers_to_output:
      print("{} -> {} ({}/500)".format(i,set(numbers_to_output[i]),get_set_string(set(numbers_to_output[i]))))
  #     print(i)
  #     print(set(numbers_to_output[i]))

  if USING_MNIST:
    string_to_output = {'000':0,'001':1,'010':2,'011':3,'100':4,'101':5,'110':6,'111':7}
    def get_output_count(num):
      counts = [0]*8
      for val in numbers_to_output[num]:
        counts[string_to_output[val]] += 1
      return counts
    for n in range(10):
      print(", ".join(map(lambda t: str(t),get_output_count(n))))



In [0]:
iterations = 2000
N=3




output = None
output_checkpoint = None

computed_nodes = compute_nodes(s)


utility = tf.math.log(tf.math.maximum(0.1,3+compute_utility_2(computed_nodes,1,0,-5)))
# utility = compute_utility_2(computed_nodes,1,.5,-1.3)


train_ops = []
for i in range(N+1):
  train_ops.append(tf.train.AdamOptimizer(.03).minimize(-utility[i])) #,var_list=[x_individual[i],B_individual[i]]
# train_op = tf.train.AdamOptimizer(1e-2).minimize(-tf.reduce_sum(utility))
init_op = tf.initialize_all_variables()

last_information = 0
infolog  = []
with tf.Session() as sess:
    sess.run(init_op)
#       print("\nresults:\n{}".format(sess.run(computed_nodes)))
    for i in range(iterations):
#         if i%2000 == 0:
#           print("\nweights:\n{}".format(sess.run(x)))
#           print("\nbiases:\n{}".format(sess.run(B)))
      if i%20==0:
        output = sess.run(computed_nodes)
        infolog.append(get_info(output))
      if i%20==0:
        output = sess.run(computed_nodes)
        print("trial {}: utility = {}, information={}".format(i,sess.run(utility),get_info(output)))
#         print("\nweights:\n{}".format(sess.run(x)))
#         print("\nbiases:\n{}".format(sess.run(B)))
        if np.isfinite(output).all():
          output_checkpoint = output

        information = get_info(output)
#         if information == last_information:
#           x = random.randint(0,2)
#           print("entering fixit mode: Node {} selected".format(x+1))
#           for k in range(100):
#              sess.run(train_ops[x])
#           information = get_info(output)
#           print("ending, new info = {}".format(information))

        last_information = information
#           dotheprint(output_checkpoint)


      for node in range(N+1):
        sess.run(train_ops[node])
#       sess.run(train_op)""

      if i % 50 == 0 and not np.isfinite(output).all():
        break


#       print("\nimages:\n{}".format(sess.run(s)))
#       print("\nweights:\n{}".format(sess.run(x)))
#       print("\nbiases:\n{}".format(sess.run(B)))
#       print("\nresults:\n{}".format(sess.run(computed_nodes)))
    output = sess.run(computed_nodes)


# vals = [do_trial_run() for sdfsd in range(20)]
# print(get_utility_after_training(.2, .1))


In [0]:
print(x_train)

In [0]:
def get_set_string(numset):
  summation = 0
  for value in numset:
    summation += c2[value]
  return summation

#computing the maximum information
counter = {}
for image in x_train:
  key = tuple(image)
  if key in counter:
    counter[key] += 1
  else:
    counter[key] = 1
information = 0
for key, val in counter.items():
  p = float(val)/float(data_size)
  information -= p*math.log(p)/math.log(2)
# print(information)
c2 = {}
for key,val in counter.items():
  realnumb = key[0]*32+key[1]*16+key[2]*8+key[3]*4+key[4]*2+key[5]
#   print("{}: {}".format(realnumb,val))
  if realnumb not in c2:
    c2[realnumb] = val
  else:
    c2[realnumb] += val
numbers_to_output = {}
for i in range(len(x_train)):

  num = x_train[i][0]*32+x_train[i][1]*16+x_train[i][2]*8+x_train[i][3]*4+x_train[i][4]*2+x_train[i][5]
  o = list(map(lambda x: int(x),list(get_output(output[i]))))
  o = 8*o[0]+4*o[1]+2*o[2]+o[3]
  num,o=o,num
  tmp0 = numbers_to_output.get(num,[])
  tmp0.append(o)
  numbers_to_output[num] = tmp0
variouscounts = []
for i in range(16):
  if i in numbers_to_output:
    cnt = get_set_string(set(numbers_to_output[i]))
    variouscounts.append(cnt)
    print("{} -> {} ({}/500)".format(i,set(numbers_to_output[i]),cnt))

print(get_info(output))
print("mean: {}, var: {}".format(np.mean(variouscounts), np.var(variouscounts)))

0 -> {56, 60, 28, 44} (11/500)
1 -> {32, 40, 41, 10, 42} (17/500)
2 -> {9, 24, 57, 27, 25} (10/500)
3 -> {11, 43, 59} (7/500)
4 -> {13, 61, 45, 15} (6/500)
5 -> {37} (6/500)
6 -> {17, 21, 53, 31, 29, 63} (15/500)
7 -> {49, 35, 33} (6/500)
8 -> {14, 62, 30} (7/500)
9 -> {0, 34, 46} (7/500)
10 -> {58, 16, 48, 18, 26} (13/500)
11 -> {50, 2} (8/500)
12 -> {4, 20, 54, 52} (14/500)
13 -> {36, 38, 39} (12/500)
14 -> {22} (1/500)
15 -> {19, 51, 55, 7} (10/500)
3.8521883533783736
mean: 9.375, var: 16.109375


In [0]:
dotheprint(output)
import matplotlib.pyplot as plt
plt.plot(infolog, label = "information")
plt.plot([4.0 for sdfasds in range(len(infolog))], label = "maximum information")
plt.xlabel("trial / 20")
plt.ylabel("information")
plt.show()


# from scipy.ndimage.filters import gaussian_filter

# blurred = gaussian_filter(infolog, sigma=20)
# plt.plot(blurred)
# plt.show()

TypeError: ignored

Making Map after some trials of the resulting information

In [0]:
import pylab as plt
import numpy as np

# Sample data
side = np.linspace(-5,7,20,20)
lambda_plot,kappa_plot = np.meshgrid(side,side)
Z = np.zeros((len(lambda_plot),len(lambda_plot[0])))


In [0]:
for x_p in range(len(lambda_plot)):
  print(x_p)
  for y_p in range(len(lambda_plot[0])):
      l = lambda_plot[x_p][y_p]
      k = kappa_plot[x_p][y_p]
#     
      Z[x_p][y_p] = sum([get_utility_after_training(l, k) for afsasdf in range(4)])/4.0

# Plot the density map using nearest-neighbor interpolation
plt.pcolormesh(lambda_plot,kappa_plot,Z)
plt.show()

0
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


NameError: ignored

In [0]:
print(Z)
print(lambda_plot)

NameError: ignored

In [0]:

dotheprint()

# output = output_checkpoint
# get_info(output)

# numbers_to_output = {}
# for i in range(len(x_train)):
#   if USING_MNIST:
#     num = y_train[i]
#   else:
#     num = x_train[i][0]*8+x_train[i][1]*4+x_train[i][2]*2+x_train[i][3]
# #     print(num)

#   o = get_output(output[i])
# #   print("{}->{}".format(num,output))
#   s = numbers_to_output.get(num,[])
#   s.append(o)
#   numbers_to_output[num] = s
# for i in range(16):
#   if i in numbers_to_output:
#     print(i)
#     print(set(numbers_to_output[i]))

# if USING_MNIST:
#   string_to_output = {'000':0,'001':1,'010':2,'011':3,'100':4,'101':5,'110':6,'111':7}
#   def get_output_count(num):
#     counts = [0]*8
#     for val in numbers_to_output[num]:
#       counts[string_to_output[val]] += 1
#     return counts
#   for n in range(10):
#     print(", ".join(map(lambda t: str(t),get_output_count(n))))

In [0]:
# output = output_checkpoint
# print(output)
# print(get_info(output)) 



numbs = {}
for a in range(2):
  for b in range(2):
    for c in range(2):
      numbs[str(a)+str(b)+str(c)] = set()
for i in range(len(output)):
  key = get_output(output[i])
  value = x_train[i][0]*8+x_train[i][1]*4+x_train[i][2]*2+x_train[i][3]
#   value = np.argmax(x_train[i])
#   if value == 15:
#     print()
#     print(output[i])
#     print(x_train[i])
  numbs[key].add(value)

    
print(numbs)

In [0]:
# |